# Smartlease Hybrid Search System

# Take User Input:

* Prompts the user to enter a property search query (example: "Looking for a 2BHK in Boston under $3000").

# Refine Query (OpenAI GPT-4):

* Uses GPT-4 to rephrase the query into a cleaner, semantically rich version for better embedding.

# Extract Address (Regex):

* Optionally pulls out the address part from the query (if available), used later for result context.

# Generate Embedding:

* Sends the refined query to OpenAI's text-embedding-ada-002 model to get a vector representation.

# Semantic Search (Pinecone):

* Uses the embedding to search the Pinecone vector DB for top-matching property embeddings.

* Returns metadata of top matches.

# Post-Filter & Format (OpenAI GPT-4):

* Sends search results and query context to GPT-4.

* GPT-4 filters and formats the matches into structured JSON.

# Store Output in JSON:

* Saves the final results (with original query + metadata) to a file query_metadata_log.json.



In [61]:
import openai
import json
import os

# Set your OpenAI API key
openai.api_key = "" 

def extract_metadata_from_query(user_query: str) -> dict:
    system_prompt = """
You are an intelligent assistant that extracts structured metadata from real estate search queries.

Extract only the fields that are mentioned in the user query. Return the output in JSON format.

Valid metadata fields:
- PROPERTY_ID
- PROPERTY_TITLE
- PROPERTY_ADDRESS
- MONTHLY_RENT
- BEDROOMS
- BATHROOMS
- SQUARE_FEET
- AVAILABILITY
- DEPOSIT
- LEASE_DURATION
- HOUSE_FEATURES
- UTILITIES_INCLUDED
- ABOUT_PROPERTY_DETAILS
- NEIGHBORHOOD_DETAILS
- ADDITIONAL_PROPERTY_DETAILS
- PROPERTY_IMAGE_FOLDER_S3_PATH
"""

    user_prompt = f"User Query: {user_query}\n\nReturn metadata in JSON format."

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",  # or gpt-3.5-turbo
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.2
        )

        raw_content = response["choices"][0]["message"]["content"]
        metadata_json = json.loads(raw_content)

        return {
            "user_query": user_query,
            "metadata_extracted": metadata_json
        }

    except json.JSONDecodeError:
        return {
            "error": "Could not parse response as JSON",
            "raw_response": raw_content
        }
    except Exception as e:
        return {
            "error": str(e)
        }


def save_metadata_to_json(output: dict, filepath: str = "query_metadata_log.json"):
    # Check if file exists; if not, create a new list
    if os.path.exists(filepath):
        with open(filepath, "r") as f:
            try:
                existing_data = json.load(f)
            except json.JSONDecodeError:
                existing_data = []
    else:
        existing_data = []

    # Append new output
    existing_data.append(output)

    # Write back to file
    with open(filepath, "w") as f:
        json.dump(existing_data, f, indent=2)

    print(f"Metadata saved to {filepath}")



# Prompt user input
if __name__ == "__main__":
    # print("Welcome to SmartLease Property Finder!")
    query = input("Please enter your property requirement, like: 2-bedroom apartment in Boston under $3000 with parking'):\n> ")
    result = extract_metadata_from_query(query)
    print("Extracted Metadata:")
    print(json.dumps(result, indent=2))

    #Save metadata to query_metadata_log.json
    save_metadata_to_json(result)
    # print("Metadata saved to query_metadata_log.json")





Please enter your property requirement, like: 2-bedroom apartment in Boston under $3000 with parking'):
>  Give me properties with 5 bedrooms and rent can be 5200  and is closest to Wentworth Institute of Technology based on drive distance in miles


Extracted Metadata:
{
  "user_query": "Give me properties with 5 bedrooms and rent can be 5200  and is closest to Wentworth Institute of Technology based on drive distance in miles",
  "metadata_extracted": {
    "BEDROOMS": 5,
    "MONTHLY_RENT": 5200,
    "PROPERTY_ADDRESS": "Wentworth Institute of Technology"
  }
}
Metadata saved to query_metadata_log.json


# Connect to Snowflake DB and test connection

In [ ]:
# ! pip install snowflake-connector-python

In [63]:
import snowflake.connector

# Snowflake credentials
SNOWFLAKE_ACCOUNT = "SFEDU02-PDB57018"
SNOWFLAKE_USER = "BADGER"
# Not needed with externalbrowser authentication
SNOWFLAKE_PASSWORD = ""  
SNOWFLAKE_AUTHENTICATOR = "externalbrowser"
SNOWFLAKE_ROLE = "TRAINING_ROLE"
SNOWFLAKE_DATABASE = "LISTINGS" 
SNOWFLAKE_SCHEMA = "PUBLIC"    
SNOWFLAKE_WAREHOUSE = "LISTINGS_WH" 

# Connect to Snowflake
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
)

# Create a cursor to run queries
cursor = conn.cursor()

# Test query
try:
    cursor.execute("SELECT * FROM PROPERTIES_WITH_EMBEDDINGS LIMIT 1;")

    result = cursor.fetchall()
    print("Connection successful! Sample row:\n")

    print(result)

except Exception as e:
    print("Connection failed:", str(e))
finally:
    cursor.close()
    conn.close()


Connection successful! Sample row:

[('property_1', '29 woodford st', '29 woodford st, boston, ma 02125', 'rent: $5,000', '4 bedrooms', '3 bathrooms', '2,000 square feet', 'available now', None, None, 'amenities: kitchen, refrigerator', None, "this spacious townhouse offers 3 bedrooms, 2.5 bathrooms, and 2 garage spaces. the home features a private deck and a separate balcony for outdoor enjoyment. the master suite, located on the top floor, includes a walk-in closet, double vanity, standing glass shower, and a ceramic tub. built-in furniture designs throughout provide ready spaces for electronics, with a built-in desk area along the walkway. located in a prime boston area, just steps from upham's corner station, this home offers easy access to downtown boston, northeastern university, and massart at ruggles station.in addition to public schools, nearby private school options include the epiphany school and boston college high school. you ll also enjoy proximity to shopping centers, in

# Hybrid Snowflake Search

In [78]:
import snowflake.connector
import json

# Load latest metadata entry from local JSON
with open("query_metadata_log.json", "r") as f:
    data = json.load(f)

latest = data[-1]
user_query = latest["user_query"]
metadata_filters = latest["metadata_extracted"]

# Sanitize user query for embedding call inside SQL
user_query_safe = user_query.replace("'", "''")

# Connect to Snowflake
conn = snowflake.connector.connect(
    account="SFEDU02-PDB57018",
    user="BADGER",
    password="",
    role="TRAINING_ROLE",
    warehouse="LISTINGS_WH",
    database="LISTINGS",
    schema="PUBLIC"
)
cur = conn.cursor()

# Build metadata filter WHERE clause
filter_clauses = []
for key, value in metadata_filters.items():
    if value:
        if isinstance(value, (int, float)):
            filter_clauses.append(f"{key} = {value}")
        elif isinstance(value, str) and value.strip().startswith("<"):
            num = value.strip("<$ ")
            filter_clauses.append(f"TRY_TO_NUMBER(REGEXP_REPLACE({key}, '[^0-9.]', '')) < {num}")
        elif isinstance(value, str) and value.strip().startswith(">"):
            num = value.strip(">$ ")
            filter_clauses.append(f"TRY_TO_NUMBER(REGEXP_REPLACE({key}, '[^0-9.]', '')) > {num}")
        elif isinstance(value, bool):
            filter_clauses.append(f"{key} = {str(value).upper()}")
        else:
            filter_clauses.append(f"{key} ILIKE '%{value}%'")

where_clause = " AND ".join(filter_clauses)
if where_clause:
    where_clause = f"WHERE {where_clause}"

# SQL with filters (hybrid search)
sql_with_filters = f"""
SELECT *,
  VECTOR_COSINE_SIMILARITY(
    PROPERTIES_EMBEDDING,
    SNOWFLAKE.CORTEX.EMBED_TEXT_1024('snowflake-arctic-embed-l-v2.0', '{user_query_safe}')
  ) AS similarity
FROM PROPERTIES_WITH_EMBEDDINGS
{where_clause}
ORDER BY similarity DESC
LIMIT 10;
"""

print("Executing hybrid query with filters...\n")
print(sql_with_filters)

# Run hybrid query
cur.execute(sql_with_filters)
results = cur.fetchall()

# If no results, fallback to semantic-only
if not results:
    print("\nNo results found with metadata filters. Trying semantic-only fallback...\n")

    sql_no_filters = f"""
    SELECT *,
      VECTOR_COSINE_SIMILARITY(
        PROPERTIES_EMBEDDING,
        SNOWFLAKE.CORTEX.EMBED_TEXT_1024('snowflake-arctic-embed-l-v2.0', '{user_query_safe}')
      ) AS similarity
    FROM PROPERTIES_WITH_EMBEDDINGS
    ORDER BY similarity DESC
    LIMIT 10;
    """

    print("Fallback Query:\n", sql_no_filters)
    cur.execute(sql_no_filters)
    results = cur.fetchall()

# Safely fetch columns after query execution
columns = [col[0] for col in cur.description]

# Show results
if not results:
    print("\nNo matching properties found even after fallback.\n")
else:
    print("\nTop matching properties:\n")
    for row in results:
        print(dict(zip(columns, row)))

cur.close()
conn.close()


🔍 Executing hybrid query with filters...


SELECT *,
  VECTOR_COSINE_SIMILARITY(
    PROPERTIES_EMBEDDING,
    SNOWFLAKE.CORTEX.EMBED_TEXT_1024('snowflake-arctic-embed-l-v2.0', 'Give me properties with 5 bedrooms and rent can be 5200  and is closest to Wentworth Institute of Technology based on drive distance in miles')
  ) AS similarity
FROM PROPERTIES_WITH_EMBEDDINGS
WHERE BEDROOMS ILIKE '%5 BEDROOMS%' AND MONTHLY_RENT ILIKE '%Rent: $5,200%' AND PROPERTY_ADDRESS ILIKE '%Wentworth Institute of Technology%'
ORDER BY similarity DESC
LIMIT 10;


⚠️ No results found with metadata filters. Trying semantic-only fallback...

🔁 Fallback Query:
 
    SELECT *,
      VECTOR_COSINE_SIMILARITY(
        PROPERTIES_EMBEDDING,
        SNOWFLAKE.CORTEX.EMBED_TEXT_1024('snowflake-arctic-embed-l-v2.0', 'Give me properties with 5 bedrooms and rent can be 5200  and is closest to Wentworth Institute of Technology based on drive distance in miles')
      ) AS similarity
    FROM PROPERTIES_WITH_EMBEDDINGS


In [84]:
import snowflake.connector
import json

# Load latest metadata entry from local JSON
with open("query_metadata_log.json", "r") as f:
    data = json.load(f)

latest = data[-1]
user_query = latest["user_query"]
metadata_filters = latest["metadata_extracted"]

# Sanitize user query for embedding call inside SQL
user_query_safe = user_query.replace("'", "''")

# Connect to Snowflake
conn = snowflake.connector.connect(
    account="SFEDU02-PDB57018",
    user="BADGER",
    password="",
    role="TRAINING_ROLE",
    warehouse="LISTINGS_WH",
    database="LISTINGS",
    schema="PUBLIC"
)
cur = conn.cursor()

# Build metadata filter WHERE clause
filter_clauses = []
for key, value in metadata_filters.items():
    if value:
        if isinstance(value, (int, float)):
            filter_clauses.append(f"{key} = {value}")
        elif isinstance(value, str) and value.strip().startswith("<"):
            num = value.strip("<$ ")
            filter_clauses.append(f"TRY_TO_NUMBER(REGEXP_REPLACE({key}, '[^0-9.]', '')) < {num}")
        elif isinstance(value, str) and value.strip().startswith(">"):
            num = value.strip(">$ ")
            filter_clauses.append(f"TRY_TO_NUMBER(REGEXP_REPLACE({key}, '[^0-9.]', '')) > {num}")
        elif isinstance(value, bool):
            filter_clauses.append(f"{key} = {str(value).upper()}")
        else:
            filter_clauses.append(f"{key} ILIKE '%{value}%'")

where_clause = " AND ".join(filter_clauses)
if where_clause:
    where_clause = f"WHERE {where_clause}"

# SQL with metadata filters + semantic embedding
sql_with_filters = f"""
SELECT *,
  VECTOR_COSINE_SIMILARITY(
    PROPERTIES_EMBEDDING,
    SNOWFLAKE.CORTEX.EMBED_TEXT_1024('snowflake-arctic-embed-l-v2.0', '{user_query_safe}')
  ) AS similarity
FROM PROPERTIES_WITH_EMBEDDINGS
{where_clause}
ORDER BY similarity DESC
LIMIT 10;
"""

print("Executing hybrid query with filters\n")
print(sql_with_filters)

cur.execute(sql_with_filters)
results = cur.fetchall()

# Fallback if no results with metadata filters
if not results:
    print("\nNo results found with metadata filters. Trying semantic-only fallback\n")

    sql_no_filters = f"""
    SELECT *,
      VECTOR_COSINE_SIMILARITY(
        PROPERTIES_EMBEDDING,
        SNOWFLAKE.CORTEX.EMBED_TEXT_1024('snowflake-arctic-embed-l-v2.0', '{user_query_safe}')
      ) AS similarity
    FROM PROPERTIES_WITH_EMBEDDINGS
    ORDER BY similarity DESC
    LIMIT 10;
    """

    print("🔁 Fallback Query:\n", sql_no_filters)
    cur.execute(sql_no_filters)
    results = cur.fetchall()

# Get columns after execution
columns = [col[0] for col in cur.description]

# Only show selected fields
selected_fields = [
    "PROPERTY_ID",
    "PROPERTY_TITLE",
    "PROPERTY_ADDRESS",
    "MONTHLY_RENT",
    "BEDROOMS",
    "BATHROOMS",
    "SQUARE_FEET",
    "AVAILABILITY",
    "DEPOSIT",
    "LEASE_DURATION",
    "HOUSE_FEATURES",
    "UTILITIES_INCLUDED",
    "similarity"
]

# Display
if not results:
    print("\nNo matching properties found even after fallback.\n")
else:
    print("\nTop matching properties:\n")
    for row in results:
        record = dict(zip(columns, row))
        output = {key: record.get(key) for key in selected_fields}
        print(json.dumps(output, indent=2))

cur.close()
conn.close()


Executing hybrid query with filters


SELECT *,
  VECTOR_COSINE_SIMILARITY(
    PROPERTIES_EMBEDDING,
    SNOWFLAKE.CORTEX.EMBED_TEXT_1024('snowflake-arctic-embed-l-v2.0', 'Give me properties with 5 bedrooms and rent can be 5200  and is closest to Wentworth Institute of Technology based on drive distance in miles')
  ) AS similarity
FROM PROPERTIES_WITH_EMBEDDINGS
WHERE BEDROOMS ILIKE '%5 BEDROOMS%' AND MONTHLY_RENT ILIKE '%Rent: $5,200%' AND PROPERTY_ADDRESS ILIKE '%Wentworth Institute of Technology%'
ORDER BY similarity DESC
LIMIT 10;


No results found with metadata filters. Trying semantic-only fallback

🔁 Fallback Query:
 
    SELECT *,
      VECTOR_COSINE_SIMILARITY(
        PROPERTIES_EMBEDDING,
        SNOWFLAKE.CORTEX.EMBED_TEXT_1024('snowflake-arctic-embed-l-v2.0', 'Give me properties with 5 bedrooms and rent can be 5200  and is closest to Wentworth Institute of Technology based on drive distance in miles')
      ) AS similarity
    FROM PROPERTIES_WITH_EMBEDDINGS
    ORDER B